# INSTALLING LIBRARIES

In [26]:
!pip install parfit
!pip install scikit-plot
!pip install contractions
!pip install wordcloud
! pip install --upgrade tensorflow

  Using cached parfit-0.220.tar.gz (5.4 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [27]:
## Import the relevent libraries 

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import re
from sklearn.pipeline import Pipeline
import pickle
import nltk
import string

from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
from nltk.tokenize import word_tokenize, TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

import tensorflow as tf

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from matplotlib import pyplot as plt
from random import randint
from tensorflow.keras.utils import to_categorical

# LOADING DATA

In [3]:
## Load Data

train_df = pd.read_csv(r'C:\Users\Prince.Maponyane\Downloads\AI Explore\6. Classification\13 Hackathon\train_set.csv')
test_df = pd.read_csv(r'C:\Users\Prince.Maponyane\Downloads\AI Explore\6. Classification\13 Hackathon\test_set.csv')


In [4]:
## Check that it loaded

train_df.head()


,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


In [5]:
## Check that it loaded

test_df.head()

,index,text
0,1,"Mmasepala, fa maemo a a kgethegileng a letlele..."
1,2,Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,Tshivhumbeo tshi fana na ngano dza vhathu.
3,4,Kube inja nelikati betingevakala kutsi titsini...
4,5,Winste op buitelandse valuta.


# CREATE DATAFRAME COPIES

In [6]:
## Create copies of the df's

train = train_df
test = test_df

# CLEAN DATA

In [7]:
## See the punctuations that will be removed 

print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [8]:
## Remove Punctuation 

def remove_punctuation(text):
    return ''.join([l for l in text if l not in string.punctuation])

In [9]:
train['text'] = train['text'].apply(remove_punctuation)
train['text'].iloc[:]

0        umgaqosiseko wenza amalungiselelo kumaziko axh...
1        idha iya kuba nobulumko bokubeka umsebenzi nap...
2        the province of kwazulunatal department of tra...
3        o netefatša gore o ba file dilo ka moka tše le...
4        khomishini ya ndinganyiso ya mbeu yo ewa maana...
                               ...                        
32995    popo ya dipolateforomo tse ke go tlisa boetele...
32996    modise mosadi na o ntse o sa utlwe hore thaban...
32997    closing date for the submission of completed t...
32998    nawuphina umntu ofunyenwe enetyala phantsi kwa...
32999    mafapha a mang le ona a lokela ho etsa ditlale...
Name: text, Length: 33000, dtype: object

In [10]:
test['text'] = test['text'].apply(remove_punctuation)
test['text'].iloc[:]

0       Mmasepala fa maemo a a kgethegileng a letlelel...
1       Uzakwaziswa ngokufaneleko nakungafuneka eminye...
2               Tshivhumbeo tshi fana na ngano dza vhathu
3       Kube inja nelikati betingevakala kutsi titsini...
4                            Winste op buitelandse valuta
                              ...                        
5677                      You mark your ballot in private
5678    Ge o ka kgetha ka bowena go se šomiše Mofani k...
5679    E Ka kopo etsa kgetho ya hao ka hloko hobane h...
5680    TB ke bokudi ba PMB mme Morero o tla lefella t...
5681                  Vakatjhela iwebhusayidi yethu kuwww
Name: text, Length: 5682, dtype: object

In [11]:
## Lower Case

train['text'] = train['text'].str.lower()

In [12]:
test['text'] = test['text'].str.lower()

In [13]:
test

,index,text
0,1,mmasepala fa maemo a a kgethegileng a letlelel...
1,2,uzakwaziswa ngokufaneleko nakungafuneka eminye...
2,3,tshivhumbeo tshi fana na ngano dza vhathu
3,4,kube inja nelikati betingevakala kutsi titsini...
4,5,winste op buitelandse valuta
...,...,...
5677,5678,you mark your ballot in private
5678,5679,ge o ka kgetha ka bowena go se šomiše mofani k...
5679,5680,e ka kopo etsa kgetho ya hao ka hloko hobane h...
5680,5681,tb ke bokudi ba pmb mme morero o tla lefella t...


In [14]:
## To improve efficiency - tokenize

tokeniser = TreebankWordTokenizer()
train['tokens'] = train['text'].apply(tokeniser.tokenize)
train['tokens'].iloc[:]

0        [umgaqosiseko, wenza, amalungiselelo, kumaziko...
1        [idha, iya, kuba, nobulumko, bokubeka, umseben...
2        [the, province, of, kwazulunatal, department, ...
3        [o, netefatša, gore, o, ba, file, dilo, ka, mo...
4        [khomishini, ya, ndinganyiso, ya, mbeu, yo, ew...
                               ...                        
32995    [popo, ya, dipolateforomo, tse, ke, go, tlisa,...
32996    [modise, mosadi, na, o, ntse, o, sa, utlwe, ho...
32997    [closing, date, for, the, submission, of, comp...
32998    [nawuphina, umntu, ofunyenwe, enetyala, phants...
32999    [mafapha, a, mang, le, ona, a, lokela, ho, ets...
Name: tokens, Length: 33000, dtype: object

In [15]:
tokeniser = TreebankWordTokenizer()
test['tokens'] = test['text'].apply(tokeniser.tokenize)
test['tokens'].iloc[:]

0       [mmasepala, fa, maemo, a, a, kgethegileng, a, ...
1       [uzakwaziswa, ngokufaneleko, nakungafuneka, em...
2       [tshivhumbeo, tshi, fana, na, ngano, dza, vhathu]
3       [kube, inja, nelikati, betingevakala, kutsi, t...
4                       [winste, op, buitelandse, valuta]
                              ...                        
5677               [you, mark, your, ballot, in, private]
5678    [ge, o, ka, kgetha, ka, bowena, go, se, šomiše...
5679    [e, ka, kopo, etsa, kgetho, ya, hao, ka, hloko...
5680    [tb, ke, bokudi, ba, pmb, mme, morero, o, tla,...
5681             [vakatjhela, iwebhusayidi, yethu, kuwww]
Name: tokens, Length: 5682, dtype: object

In [16]:
## Lemmatize

nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def lemmatize_sentence(text):
    tokens = word_tokenize(text)
    lemmatized_text = " ".join([lemmatizer.lemmatize(token) for token in tokens])
    return lemmatized_text

# Apply lemmatization to train and test DataFrames

train['lemmatized_text'] = train['text'].apply(lemmatize_sentence)

print(train)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Prince.Maponyane\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Prince.Maponyane\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


      lang_id                                               text  \
0         xho  umgaqosiseko wenza amalungiselelo kumaziko axh...   
1         xho  idha iya kuba nobulumko bokubeka umsebenzi nap...   
2         eng  the province of kwazulunatal department of tra...   
3         nso  o netefatša gore o ba file dilo ka moka tše le...   
4         ven  khomishini ya ndinganyiso ya mbeu yo ewa maana...   
...       ...                                                ...   
32995     tsn  popo ya dipolateforomo tse ke go tlisa boetele...   
32996     sot  modise mosadi na o ntse o sa utlwe hore thaban...   
32997     eng  closing date for the submission of completed t...   
32998     xho  nawuphina umntu ofunyenwe enetyala phantsi kwa...   
32999     sot  mafapha a mang le ona a lokela ho etsa ditlale...   

                                                  tokens  \
0      [umgaqosiseko, wenza, amalungiselelo, kumaziko...   
1      [idha, iya, kuba, nobulumko, bokubeka, umseben...   
2  

In [17]:
test['lemmatized_text'] = test['text'].apply(lemmatize_sentence)

print(test)

      index                                               text  \
0         1  mmasepala fa maemo a a kgethegileng a letlelel...   
1         2  uzakwaziswa ngokufaneleko nakungafuneka eminye...   
2         3          tshivhumbeo tshi fana na ngano dza vhathu   
3         4  kube inja nelikati betingevakala kutsi titsini...   
4         5                       winste op buitelandse valuta   
...     ...                                                ...   
5677   5678                    you mark your ballot in private   
5678   5679  ge o ka kgetha ka bowena go se šomiše mofani k...   
5679   5680  e ka kopo etsa kgetho ya hao ka hloko hobane h...   
5680   5681  tb ke bokudi ba pmb mme morero o tla lefella t...   
5681   5682                vakatjhela iwebhusayidi yethu kuwww   

                                                 tokens  \
0     [mmasepala, fa, maemo, a, a, kgethegileng, a, ...   
1     [uzakwaziswa, ngokufaneleko, nakungafuneka, em...   
2     [tshivhumbeo, tshi, fana

# MODELLING

In [18]:
## Assigning X,y

X = train['text']
y = train['lang_id']


In [19]:
## Train Test Split

##X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
### Initializing the TF-IDF vectorizer

def preprocess_data(train, test):

    vectorizer = TfidfVectorizer()

## Fitting the vectorizer on the training data
    
    vectorizer.fit(train['text'])

# Transforming the training and test data using the fitted vectorizer
    
    train_features = vectorizer.transform(train['text'])
    test_features = vectorizer.transform(test['text'])

    return train_features, test_features, vectorizer

In [21]:
train_features, test_features, vectorizer = preprocess_data(train, test)

In [31]:
## Logistic Regression Model

X_train, X_test, y_train, y_test = train_test_split(train_features, train['lang_id'], test_size=0.2, random_state=42)

lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

lr_preds = lr_model.predict(X_test)
lr_f1 = f1_score(y_test, lr_preds, average='weighted')

print("Logistic Regression F1 Score:", lr_f1)

Logistic Regression F1 Score: 0.9943976987335509


C:\Users\Prince.Maponyane\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [23]:
# K Nearest Neighbours

knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

knn_preds = knn_model.predict(X_test)

knn_f1 = f1_score(y_test, knn_preds, average='weighted')

print("KNN F1 Score:", knn_f1)

KNN F1 Score: 0.9620927040666665


In [24]:
## Support Vector Machine

svm = SVC()
svm.fit(X_train, y_train)

svm_predictions = svm.predict(X_test)
svm_f1 = f1_score(y_test, svm_predictions, average='weighted')

print("SVM F1 Score:", svm_f1)

SVM F1 Score: 0.9942548884498323


In [25]:
## Naive Bayes

nb = MultinomialNB()
nb.fit(X_train, y_train)

nb_predictions = nb.predict(X_test)
nb_f1 = f1_score(y_test, nb_predictions, average='weighted')

print("Naive Bayes F1 Score:", nb_f1)

Naive Bayes F1 Score: 0.9984848537376715


## Generate Predictions

# Converting the test data into TF-IDF vectors
X_test = vectorizer.transform(test['text'])

# Generating predictions on the best performing model
test_predictions = nb.predict(X_test)

# Creating a submission dataframe with 'index' and 'lang_id' columns

submission_df = pd.DataFrame({'index': test['index'], 'lang_id': test_predictions})

submission_df.to_csv('FinalSub1.csv', index=False)